In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corpus-eng/corpus-engineering.json


In [2]:
# Cell 2: Install Libraries (VERBOSE)
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True' # <--- ADD THIS LINE
print("Installing libraries with verbose output...")

print("\nInstalling hazm...")
!pip install hazm # For Persian NLP (normalization, tokenization)

print("\nInstalling sentence-transformers...")
!pip install sentence-transformers # For embedding models

print("\nInstalling chromadb...")
!pip install chromadb # For the vector database

print("\nInstalling tqdm...")
!pip install tqdm # For progress bars

#!pip install faiss-gpu

print("\nLibrary installation commands executed.")

Installing libraries with verbose output...

Installing hazm...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.1 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=ef428ad44fbe9b8fcb7c762423910e07ca9043645dcc71a8fc0500acbc7fdf12
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uni

In [3]:
import time # <<< ADD THIS IMPORT
# import faiss # No longer needed
import chromadb # Import chromadb
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from tqdm.auto import tqdm # For progress bars
import gc # For garbage collection
import torch # Import torch to check for GPU

2025-05-08 08:35:09.745333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746693309.920219      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746693309.971328      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import json
import os

def load_and_prepare_documents(filepath):
    """Loads documents from JSON, combines title and abstract."""
    documents = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for doc in data:
            doc_id = str(doc.get('id', '')) # Ensure ID is a string for ChromaDB
            title = doc.get('title', '')
            abstract = doc.get('abstract', '')
            # Combine title and abstract, adding a separator for clarity
            combined_text = f"{title}\n\n{abstract}".strip()
            if doc_id and combined_text: # Only add if ID and text exist
                documents.append({
                    'id': doc_id,
                    'text': combined_text,
                    # Keep original fields if needed later, e.g., for metadata
                    'metadata': {
                        'original_title': title,
                        'first_subject': doc.get('FirstSubject', ''),
                        'second_subject': doc.get('SecondSubject', '')
                    }
                })
            else:
                print(f"Warning: Skipping document due to missing ID or text. Original data: {doc}")
        print(f"Loaded {len(documents)} documents from '{filepath}'.")
        return documents
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {filepath}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred during loading: {e}")
        return []

# Specify the path to your JSON file in Kaggle's directory structure
json_filename = "/kaggle/input/corpus-eng/corpus-engineering.json"

# Verify the file exists
if not os.path.exists(json_filename):
    raise ValueError(f"File not found: {json_filename}")
    
# Load the data from the file
corpus = load_and_prepare_documents(json_filename)

# Optional: Display the first loaded document to check
if corpus:
    print("\n--- Example Document (First one) ---")
    print(f"ID: {corpus[0]['id']}")
    print(f"Combined Text:\n{corpus[0]['text']}")
    print(f"Metadata: {corpus[0]['metadata']}")

Loaded 10846 documents from '/kaggle/input/corpus-eng/corpus-engineering.json'.

--- Example Document (First one) ---
ID: 802871
Combined Text:
شناسایی عوامل بیماری‌زای قارچی گیاهان دارویی کشت شده در استان کرمان

گیاهان دارویی در طول تاریخ همیشه با انسان قرابت خاصی داشته و آثار دارویی آن بر هیچ کس پوشیده نیست. گیاهان دارویی گیاهانی هستند که یک یا برخی از اندام های آن حاوی ماده موثره است این ماده که کمتر از یک درصد وزن خشک گیاه را تشکیل می دهد دارای خواص دارویی موثر بر موجودات زنده است همچنین کاشت داشت و برداشت این گیاهان به منظور استفاده از ماده موثره ی آنها انجام می گیرد و به رغم سابقه چند هزار ساله علم طب گیاهی و وجود کتب رساله ها و مقالات فراوان در این زمینه تحقیق در مورد بیماری های گیاهان دارویی قدمت زیادی ندارد به طور کلی یکی از مشکلات کشت انبوه گیاهان دارویی بیماری های مختلفی از جمله بیماری های قارچی می باشد که تحقیقات چندانی بروی آنها صورت نگرفته است و با توجه به این که بروز بیماری های گیاهان دارویی تابع شرایط محیطی می باشد ضروری است در هر منطقه تحقیقات جداگانه ای صورت بگیرد و ل

In [5]:
import re
from tqdm import tqdm
import hazm
# Initialize the Hazm Normalizer
normalizer = hazm.Normalizer()

def clean_and_normalize(text):
    """Applies Hazm normalization and basic cleaning."""
    # Apply Hazm normalizer (handles ی, ک, ZWNJ variations, etc.)
    normalized_text = normalizer.normalize(text)
    # Remove excessive whitespace (newlines, tabs, multiple spaces) -> replace with single space
    cleaned_text = re.sub(r'\s+', ' ', normalized_text).strip()
    # Optional: Add any other specific cleaning rules here if needed
    # e.g., removing specific punctuation, converting numbers, etc.
    return cleaned_text

# --- Apply cleaning to all documents ---
print("Cleaning and normalizing documents...")
cleaned_corpus = []
for doc in tqdm(corpus):
    cleaned_text = clean_and_normalize(doc['text'])
    if cleaned_text: # Only keep if text remains after cleaning
        cleaned_corpus.append({
            'id': doc['id'], # Use the original document ID as the chunk ID for now
            'text': cleaned_text,
            'metadata': doc['metadata'] # Carry over metadata
        })
print(f"\nFinished cleaning. Resulting number of documents/chunks: {len(cleaned_corpus)}")

# Optional: Display the cleaned version of the first document
if cleaned_corpus:
    print("\n--- Example Cleaned Document (First one) ---")
    print(f"ID: {cleaned_corpus[0]['id']}")
    print(f"Cleaned Text:\n{cleaned_corpus[0]['text']}")

Cleaning and normalizing documents...


100%|██████████| 10846/10846 [00:23<00:00, 454.04it/s]


Finished cleaning. Resulting number of documents/chunks: 10846

--- Example Cleaned Document (First one) ---
ID: 802871
Cleaned Text:
شناسایی عوامل بیماری‌زای قارچی گیاهان دارویی کشت شده در استان کرمان گیاهان دارویی در طول تاریخ همیشه با انسان قرابت خاصی داشته و آثار دارویی آن بر هیچ‌کس پوشیده نیست. گیاهان دارویی گیاهانی هستند که یک یا برخی از اندام‌های آن حاوی ماده موثره است این ماده که کمتر از یک درصد وزن خشک گیاه را تشکیل می‌دهد دارای خواص دارویی موثر بر موجودات زنده است همچنین کاشت داشت و برداشت این گیاهان به منظور استفاده از ماده موثره‌ی آنها انجام می‌گیرد و به رغم سابقه چند هزار ساله علم طب گیاهی و وجود کتب رساله‌ها و مقالات فراوان در این زمینه تحقیق در مورد بیماری‌های گیاهان دارویی قدمت زیادی ندارد به طور کلی یکی از مشکلات کشت انبوه گیاهان دارویی بیماری‌های مختلفی از جمله بیماری‌های قارچی می‌باشد که تحقیقات چندانی بروی آنها صورت نگرفته است و با توجه به این‌که بروز بیماری‌های گیاهان دارویی تابع شرایط محیطی می‌باشد ضروری است در هر منطقه تحقیقات جداگانه‌ای صورت بگیرد و لزوم تحقیق 

In [6]:

if 'cleaned_corpus' in globals():
    print(f"Variable 'cleaned_corpus' exists.")
    if isinstance(cleaned_corpus, list):
        print(f"It is a list with {len(cleaned_corpus)} items.")
        if len(cleaned_corpus) > 0:
            first_item = cleaned_corpus[0]
            print(f"First item type: {type(first_item)}")
            if isinstance(first_item, dict):
                print(f"First item is a dictionary with keys: {list(first_item.keys())}")
                # --- Check for the CORRECT keys ---
                if 'id' in first_item and 'text' in first_item:
                    print("Structure looks correct (has 'id' and 'text').") # Corrected message
                    print("\nFirst item example:")
                    # Print safely, handle potential missing keys if needed later
                    print(f"  id: {first_item.get('id')}")
                    print(f"  text: {first_item.get('text')[:100]}...") # Print first 100 chars of text
                    if 'metadata' in first_item:
                         print(f"  metadata keys: {list(first_item.get('metadata', {}).keys())}")
                else:
                    # This error should NOT appear now based on your previous output
                    print("Error: First item dictionary is missing 'id' or 'text' key.")
            else:
                print("Error: Items in the list are not dictionaries.")
        else:
            print("Warning: The list is empty.")
    else:
        print(f"Error: 'cleaned_corpus' is not a list, it is a {type(cleaned_corpus)}")
else:
    print("Error: Variable 'cleaned_corpus' does not exist.")

# --- End of corrected check cell ---

Variable 'cleaned_corpus' exists.
It is a list with 10846 items.
First item type: <class 'dict'>
First item is a dictionary with keys: ['id', 'text', 'metadata']
Structure looks correct (has 'id' and 'text').

First item example:
  id: 802871
  text: شناسایی عوامل بیماری‌زای قارچی گیاهان دارویی کشت شده در استان کرمان گیاهان دارویی در طول تاریخ همیشه...
  metadata keys: ['original_title', 'first_subject', 'second_subject']


In [7]:
MODEL_TO_TEST = "jinaai/jina-embeddings-v3" # Example: Replace with the model ID you want to evaluate


QA_DATASET_IDENTIFIER = "safora/persian-scientific-qa"
QA_DATASET_SPLIT = "train" # Using only the train split as specified by the user

# --- Database and Output Settings ---
#CHROMA_PERSIST_DIRECTORY = "./chroma_db" # Directory to save ChromaDB data in Kaggle
#RESULTS_CSV_FILE = "embedding_evaluation_results.csv" # File to save final results

TOP_K = 10 # Calculate recall up to K, and MRR uses this value



In [8]:
# Cell 14 (Corrected and Modified)
def load_qa_dataset(dataset_identifier, split):
    """Loads the Question Answering dataset."""
    print(f"Loading QA dataset: {dataset_identifier}, split: {split}")
    try:
        qa_dataset = load_dataset(dataset_identifier, split=split)
        required_columns = {'query', 'abstract_id'}
        if not required_columns.issubset(qa_dataset.column_names):
             print(f"Error: QA dataset missing required columns. Expected: {required_columns}. Found: {qa_dataset.column_names}")
             # Consider exiting or raising an error more formally
             # exit()
             raise ValueError(f"QA dataset missing required columns. Expected: {required_columns}. Found: {qa_dataset.column_names}")
        print(f"Loaded {len(qa_dataset)} QA pairs.")
        return qa_dataset
    except Exception as e:
        print(f"Error loading QA dataset: {e}")
        # Consider exiting or raising an error more formally
        # exit()
        raise e

def embed_texts(model, texts, batch_size=4): # <--- REDUCED BATCH SIZE FROM 8 to 4
    """Embeds a list of texts using the provided model."""
    print(f"Embedding {len(texts)} texts with batch_size={batch_size}...")
    # SentenceTransformer handles device placement (CPU/GPU) automatically
    embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True, batch_size=batch_size)
    # ChromaDB typically expects lists of floats, convert numpy arrays
    return [embedding.tolist() for embedding in embeddings]

# Note: The original notebook had the load_qa_dataset function defined twice. I've kept one.
# The IndentationError was here:
def setup_chroma_collection(client, collection_name, embeddings, ids, documents=None, metadatas=None): # Added metadatas parameter
    """Creates or gets a ChromaDB collection and adds data.""" # <--- CORRECTED INDENTATION FOR THIS DOCSTRING
    print(f"Setting up ChromaDB collection: {collection_name}")
    try:
        # Ensure collection name is valid
        if not collection_name:
             print("Error: Invalid collection name provided.")
             return None
        collection = client.get_or_create_collection(
            name=collection_name,
            metadata={"hnsw:space": "l2"} # Specify L2 distance
        )

        # Consider reducing Chroma batch size if memory is still an issue during DB population,
        # though the primary OOM is likely during embedding.
        batch_size_chroma = 1000 # Reduced from 5000, adjust if needed
        num_batches = (len(ids) + batch_size_chroma - 1) // batch_size_chroma

        print(f"Adding {len(ids)} documents to collection in {num_batches} batches of size {batch_size_chroma}...")
        for i in tqdm(range(num_batches), desc="Adding to Chroma"):
            start_idx = i * batch_size_chroma
            end_idx = min((i + 1) * batch_size_chroma, len(ids))

            batch_ids = ids[start_idx:end_idx]
            batch_embeddings = embeddings[start_idx:end_idx]
            batch_documents = documents[start_idx:end_idx] if documents else None
            batch_metadatas = metadatas[start_idx:end_idx] if metadatas else None # Add metadata batching

            if not batch_ids: # Skip if batch is empty
                continue

            collection.add(
                embeddings=batch_embeddings,
                ids=batch_ids,
                documents=batch_documents,
                metadatas=batch_metadatas # Pass metadata to ChromaDB
            )
        print("Finished adding documents to collection.")
        return collection
    except Exception as e:
        print(f"Error setting up or adding to ChromaDB collection '{collection_name}': {e}")
        # Attempt to clean up if collection was partially created
        try:
            if collection_name and client: # ensure client exists
                client.delete_collection(collection_name)
        except Exception as e_del:
            print(f"Warning during cleanup of collection {collection_name}: {e_del}")
        return None


def query_chroma_collection(collection, query_embedding, k):
    """Queries the ChromaDB collection."""
    try:
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=k
        )
        if 'ids' in results and results['ids'] and results['ids'][0]:
             retrieved_ids = results['ids'][0]
        else:
             retrieved_ids = []
        return retrieved_ids
    except Exception as e:
        print(f"Error querying ChromaDB collection: {e}")
        return []

In [9]:
# --- Main Evaluation Script ---

if __name__ == "__main__":
    # Optional: Check GPU availability
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"PyTorch is using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("PyTorch is using CPU")

    # 1. Prepare Corpus Data
    # Check for the structure produced by the user's cleaning code
    if 'cleaned_corpus' not in globals() or not isinstance(cleaned_corpus, list) or \
       not all(isinstance(doc, dict) and 'id' in doc and 'text' in doc for doc in cleaned_corpus):
        print("Error: `cleaned_corpus` variable is not defined or not in the expected format.")
        print("Expected format: [{'id': id_value, 'text': 'text...', 'metadata': {...}}, ...]")
        exit()

    # Use the keys 'text' and 'id' from the user's cleaning code
    corpus_texts = [doc['text'] for doc in cleaned_corpus]
    corpus_ids = [str(doc['id']) for doc in cleaned_corpus] # Chroma requires string IDs
    corpus_metadatas = [doc.get('metadata', {}) for doc in cleaned_corpus] # Extract metadata too
    print(f"Using {len(corpus_texts)} documents from `cleaned_corpus` variable.")

    # 2. Load QA Data
    qa_dataset = load_qa_dataset(QA_DATASET_IDENTIFIER, QA_DATASET_SPLIT)

    # --- Start Single Model Evaluation ---
    print(f"\n--- Evaluating Single Model: {MODEL_TO_TEST} ---")
    model_start_time = time.time()
    # Sanitize model name for collection name
    collection_name_suffix = MODEL_TO_TEST.replace('/', '_').replace('-', '_').replace('.', '_')
    collection_name = f"corpus_{collection_name_suffix}"
    if not collection_name:
        print("Error: Could not generate a valid collection name from the model ID.")
        exit()

    sbert_model = None
    chroma_collection = None
    chroma_client = None
    evaluation_successful = False

    try:
        # 3a. Load Model
        print("Loading embedding model...")
        sbert_model = SentenceTransformer(MODEL_TO_TEST, trust_remote_code=True)
        # Optionally force model to GPU if available
        # sbert_model.to(device) # You can leave this commented to let SentenceTransformers manage,
                                # or explicitly use sbert_model.to("cpu") if GPU memory is still an issue.

        # 3b. Embed Corpus
        # The embed_texts function (defined elsewhere) should be using a smaller batch_size
        corpus_embeddings_list = embed_texts(sbert_model, corpus_texts)

        # 3c. Setup ChromaDB Collection
        chroma_client = chromadb.Client() # In-memory client
        chroma_collection = setup_chroma_collection( # This will use the adjusted chroma batch size
            chroma_client,
            collection_name,
            corpus_embeddings_list,
            corpus_ids,
            corpus_texts,
            corpus_metadatas
        )
        # It's crucial to delete large objects like embeddings lists once they are loaded into ChromaDB
        del corpus_embeddings_list
        gc.collect()
        if torch.cuda.is_available():
           torch.cuda.empty_cache()

        if chroma_collection is None:
            raise Exception(f"Failed to setup ChromaDB collection for {MODEL_TO_TEST}.")

        # 3d. Evaluate Retrieval on QA dataset
        print(f"Evaluating retrieval on {len(qa_dataset)} QA pairs...")
        hits_at_k = {k: 0 for k in range(1, TOP_K + 1)}
        reciprocal_ranks_list = []

        all_questions_texts = [item['query'] for item in qa_dataset]
        # The embed_texts function will also use the smaller batch_size for questions
        question_embeddings_list = embed_texts(sbert_model, all_questions_texts)

        for i, qa_item in enumerate(tqdm(qa_dataset, desc="Evaluating QA pairs")):
            true_relevant_doc_id = str(qa_item['abstract_id'])
            current_query_embedding = question_embeddings_list[i]

            retrieved_doc_ids = query_chroma_collection(chroma_collection, current_query_embedding, k=TOP_K)

            found_rank = -1
            if retrieved_doc_ids:
                for k_idx, retrieved_id in enumerate(retrieved_doc_ids):
                    if retrieved_id == true_relevant_doc_id:
                        found_rank = k_idx + 1
                        break
            
            if found_rank != -1:
                 for k_val in range(found_rank, TOP_K + 1):
                     hits_at_k[k_val] += 1

            if found_rank != -1:
                reciprocal_ranks_list.append(1.0 / found_rank)
            else:
                reciprocal_ranks_list.append(0.0)
        
        # Delete question embeddings after use
        del question_embeddings_list
        gc.collect()
        if torch.cuda.is_available():
           torch.cuda.empty_cache()

        # Calculate final metrics
        num_queries = len(qa_dataset)
        final_recall_at_k = {k: (hits_at_k[k] / num_queries) if num_queries > 0 else 0 for k in range(1, TOP_K + 1)}
        mrr_score = np.mean(reciprocal_ranks_list) if reciprocal_ranks_list else 0.0

        model_end_time = time.time()
        model_eval_time = model_end_time - model_start_time

        # --- Generate Single Model Report Output ---
        print("\n--- Evaluation Report ---")
        print(f"Model Tested: {MODEL_TO_TEST}")
        print(f"Corpus Size: {len(corpus_texts)} documents")
        print(f"QA Pairs Evaluated: {num_queries}")
        print(f"Evaluation Time: {model_eval_time:.2f} seconds")
        print("--- Metrics ---")
        print(f"MRR@{TOP_K}: {mrr_score:.4f}")
        for k_val in range(1, TOP_K + 1):
             print(f"Recall@{k_val}: {final_recall_at_k[k_val]:.4f}")
        print("-------------------------")
        evaluation_successful = True

    except Exception as e:
        print(f"\n--- ERROR during evaluation for model {MODEL_TO_TEST} ---")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        # Print traceback for more details if needed
        # import traceback
        # traceback.print_exc()
        print("---------------------------------------------------------")

    finally:
        # --- Clean up resources ---
        print("\n--- Cleaning up resources ---")
        if chroma_client and collection_name:
            print(f"Attempting to delete ChromaDB collection: {collection_name}")
            try:
                chroma_client.delete_collection(collection_name)
                print("Collection deleted.")
            except Exception as e_del:
                print(f"Warning: Could not delete collection {collection_name}: {e_del}")
        # Explicitly delete objects and collect garbage
        # Check if variables exist before deleting, to prevent errors if initialization failed
        if 'sbert_model' in locals() and sbert_model is not None:
            del sbert_model
        if 'chroma_collection' in locals() and chroma_collection is not None:
            del chroma_collection
        if 'chroma_client' in locals() and chroma_client is not None:
            del chroma_client
        
        gc.collect()
        if torch.cuda.is_available():
           torch.cuda.empty_cache()
           print("CUDA cache cleared.")
        print("Cleanup finished.")

        if not evaluation_successful:
            print(f"\nEvaluation FAILED for model {MODEL_TO_TEST}.")

PyTorch is using GPU: Tesla P100-PCIE-16GB
Using 10846 documents from `cleaned_corpus` variable.
Loading QA dataset: safora/persian-scientific-qa, split: train


README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

persian-scientific-qa.json:   0%|          | 0.00/81.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/39883 [00:00<?, ? examples/s]

Loaded 39883 QA pairs.

--- Evaluating Single Model: jinaai/jina-embeddings-v3 ---
Loading embedding model...


...

Adding to Chroma: 100%|██████████| 11/11 [00:23<00:00,  2.11s/it]


Finished adding documents to collection.
Evaluating retrieval on 39883 QA pairs...
Embedding 39883 texts with batch_size=4...


Batches:   0%|          | 0/9971 [00:00<?, ?it/s]

Evaluating QA pairs: 100%|██████████| 39883/39883 [02:07<00:00, 313.07it/s]



--- Evaluation Report ---
Model Tested: jinaai/jina-embeddings-v3
Corpus Size: 10846 documents
QA Pairs Evaluated: 39883
Evaluation Time: 1453.26 seconds
--- Metrics ---
MRR@10: 0.4050
Recall@1: 0.3333
Recall@2: 0.4062
Recall@3: 0.4487
Recall@4: 0.4771
Recall@5: 0.4998
Recall@6: 0.5184
Recall@7: 0.5344
Recall@8: 0.5480
Recall@9: 0.5594
Recall@10: 0.5700
-------------------------

--- Cleaning up resources ---
Attempting to delete ChromaDB collection: corpus_jinaai_jina_embeddings_v3
Collection deleted.
CUDA cache cleared.
Cleanup finished.
